<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/09_Saving_and_Loading_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Device setting

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [3]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

# Normalize

In [4]:
x_train = x_train.reshape(-1, 28 * 28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype('float32') / 255.0

1. How to save and load model weights
2. Save and load entire model (Serializing model)
 - Save weights
 - Model architecture
 - Training Configuration (model.compile())
 - Optimizer and states

#Model

Sequential API

In [5]:
model1 = keras.Sequential(
    [
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ]
)

Functional API

In [6]:
inputs = keras.Input(784)
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10)(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

Subclassing

In [7]:
class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = layers.Dense(64, activation='relu'),
    self.dense2 = layers.Dense(10)
  
  def call(self, input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)
  
model3 = MyModel()

## 모델 weights 저장

In [8]:
model = model1

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'],
)

In [9]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 8s - loss: 0.2947 - accuracy: 0.9169 - 8s/epoch - 4ms/step
Epoch 2/2
1875/1875 - 6s - loss: 0.1430 - accuracy: 0.9585 - 6s/epoch - 3ms/step
313/313 - 1s - loss: 0.1212 - accuracy: 0.9638 - 822ms/epoch - 3ms/step


[0.12115960568189621, 0.9638000130653381]

In [10]:
model.save_weights('save_model/')

In [11]:
model = model1
model.load_weights('save_model/')
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 4s - loss: 0.1040 - accuracy: 0.9696 - 4s/epoch - 2ms/step
Epoch 2/2
1875/1875 - 4s - loss: 0.0815 - accuracy: 0.9753 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.0938 - accuracy: 0.9722 - 757ms/epoch - 2ms/step


[0.09383470565080643, 0.9721999764442444]

model1의 weights를 저장했기 때문에 model2로 load 시 경고 발생

In [12]:
model = model2
model.load_weights('save_model/')
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)


Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x7f830b8a41d0> and <keras.engine.input_layer.InputLayer object at 0x7f8300ab1190>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x7f830b8d4e90> and <keras.layers.core.dense.Dense object at 0x7f830b8a41d0>).
Epoch 1/2
1875/1875 - 4s - loss: 0.1034 - accuracy: 0.9696 - 4s/epoch - 2ms/step
Epoch 2/2
1875/1875 - 4s - loss: 0.0818 - accuracy: 0.9750 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.0935 - accuracy: 0.9726 - 763ms/epoch - 2ms/step


[0.09351453185081482, 0.972599983215332]

## 모델 전체 저장

In [13]:
model = model1

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'],
)

In [14]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 4s - loss: 0.0680 - accuracy: 0.9795 - 4s/epoch - 2ms/step
Epoch 2/2
1875/1875 - 4s - loss: 0.0551 - accuracy: 0.9828 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.0871 - accuracy: 0.9733 - 767ms/epoch - 2ms/step


[0.0871044173836708, 0.9732999801635742]

In [15]:
model.save('complete_saved_model/')

INFO:tensorflow:Assets written to: complete_saved_model/assets


모델 전체를 저장했기 때문에 불러올 때 모델에 대한 정의가 필요없음

In [16]:
model = keras.models.load_model('complete_saved_model/')

In [17]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 4s - loss: 0.0479 - accuracy: 0.9848 - 4s/epoch - 2ms/step
Epoch 2/2
1875/1875 - 4s - loss: 0.0396 - accuracy: 0.9879 - 4s/epoch - 2ms/step
313/313 - 1s - loss: 0.0866 - accuracy: 0.9746 - 759ms/epoch - 2ms/step


[0.08664020895957947, 0.9746000170707703]